In [40]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install opencv-python Pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import Label, StringVar, OptionMenu
from PIL import Image, ImageTk

# Define filter functions
def filter_original(frame):
    return frame

def filter_grayscale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

def filter_sepia(frame):
    sepia_filter = np.array([[0.272, 0.534, 0.131],
                              [0.349, 0.686, 0.168],
                              [0.393, 0.769, 0.189]])
    return cv2.transform(frame, sepia_filter)

def filter_invert(frame):
    return cv2.bitwise_not(frame)

def filter_blur(frame):
    return cv2.GaussianBlur(frame, (15, 15), 0)

def filter_edge_detection(frame):
    return cv2.Canny(frame, 100, 200)

def filter_bright(frame):
    return cv2.convertScaleAbs(frame, alpha=1.5, beta=0)

def filter_dark(frame):
    return cv2.convertScaleAbs(frame, alpha=0.5, beta=0)

def filter_contrast(frame):
    return cv2.addWeighted(frame, 1.5, np.zeros(frame.shape, frame.dtype), 0, -100)

def filter_vignette(frame):
    rows, cols = frame.shape[:2]
    X_resultant_kernel = cv2.getGaussianKernel(cols, cols / 3)
    Y_resultant_kernel = cv2.getGaussianKernel(rows, rows / 3)
    kernel = Y_resultant_kernel * X_resultant_kernel.T
    mask = kernel / kernel.max()
    return (frame * mask[..., np.newaxis]).astype(np.uint8)

def filter_cartoon(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 9, 9)
    color = cv2.bilateralFilter(frame, 9, 300, 300)
    return cv2.bitwise_and(color, color, mask=edges)

def filter_sketch(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted = cv2.bitwise_not(gray)
    blurred = cv2.GaussianBlur(inverted, (21, 21), 0)
    inverted_blurred = cv2.bitwise_not(blurred)
    return cv2.divide(gray, inverted_blurred, scale=256)

def filter_hue(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = (hsv[..., 0] + 90) % 180  # Change hue
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def filter_pencil_sketch(frame):
    gray, sketch = cv2.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
    return sketch

def filter_emboss(frame):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(frame, -1, kernel)

def filter_solarize(frame):
    return cv2.threshold(frame, 128, 255, cv2.THRESH_BINARY_INV)[1]

def filter_color_invert(frame):
    return cv2.bitwise_not(frame)

def filter_fog(frame):
    return cv2.addWeighted(frame, 0.5, np.full(frame.shape, 255, dtype=np.uint8), 0.5, 0)

def filter_pixelate(frame):
    return cv2.resize(cv2.resize(frame, (32, 32)), (640, 480), interpolation=cv2.INTER_NEAREST)

def filter_noise(frame):
    noise = np.random.normal(0, 25, frame.shape).astype(np.uint8)
    return cv2.add(frame, noise)

def filter_sharpen(frame):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(frame, -1, kernel)

def filter_cinematic(frame):
    frame = frame.astype(np.float32) / 255.0
    frame = cv2.addWeighted(frame, 1.2, np.zeros(frame.shape, frame.dtype), 0, -0.1)
    rows, cols = frame.shape[:2]
    X_resultant_kernel = cv2.getGaussianKernel(cols, cols / 3)
    Y_resultant_kernel = cv2.getGaussianKernel(rows, rows / 3)
    kernel = Y_resultant_kernel * X_resultant_kernel.T
    mask = kernel / kernel.max()
    frame = frame * mask[..., np.newaxis]
    return np.clip(frame * 255, 0, 255).astype(np.uint8)

def filter_xray(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    inverted = cv2.bitwise_not(gray)
    blurred = cv2.GaussianBlur(inverted, (15, 15), 0)
    _, xray = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    return xray

# New creative filters
def filter_posterize(frame):
    return cv2.convertScaleAbs(frame // 64 * 64)

def filter_color_pop(frame):
    # Increase saturation
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 1] = hsv[..., 1] * 1.5
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def filter_film_grain(frame):
    noise = np.random.normal(0, 25, frame.shape).astype(np.uint8)
    return cv2.add(frame, noise)

def filter_glitch(frame):
    # Randomly shift rows
    rows, cols, _ = frame.shape
    glitch_frame = frame.copy()
    for i in range(rows):
        if np.random.rand() > 0.5:
            shift = np.random.randint(-5, 5)
            glitch_frame[i] = np.roll(frame[i], shift)
    return glitch_frame

def filter_dreamy(frame):
    blurred = cv2.GaussianBlur(frame, (21, 21), 0)
    return cv2.addWeighted(frame, 0.5, blurred, 0.5, 0)

def filter_futuristic(frame):
    # Apply a color map
    return cv2.applyColorMap(frame, cv2.COLORMAP_JET)

def filter_mosaic(frame):
    return cv2.resize(cv2.resize(frame, (32, 32)), (640, 480), interpolation=cv2.INTER_NEAREST)

def filter_rainbow(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = (hsv[..., 0] + 90) % 180  # Change hue
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def filter_vintage(frame):
    # Apply sepia and add noise
    sepia = filter_sepia(frame)
    noise = np.random.normal(0, 10, sepia.shape).astype(np.uint8)
    return cv2.add(sepia, noise)

def filter_smooth(frame):
    return cv2.bilateralFilter(frame, 15, 75, 75)

def filter_lomo(frame):
    # Apply vignette and increase contrast
    vignette_frame = filter_vignette(frame)
    return cv2.addWeighted(vignette_frame, 1.5, np.zeros(frame.shape, frame.dtype), 0, -50)

def filter_saturation(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 1] = hsv[..., 1] * 2  # Increase saturation
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def filter_bokeh(frame):
    blurred = cv2.GaussianBlur(frame, (21, 21), 0)
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.circle(mask, (frame.shape[1] // 2, frame.shape[0] // 2), 100, 255, -1)
    return cv2.bitwise_and(frame, frame, mask=mask) + cv2.bitwise_and(blurred, blurred, mask=cv2.bitwise_not(mask))

def filter_silhouette(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    return cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)

def filter_frosted(frame):
    return cv2.GaussianBlur(frame, (15, 15), 0)

def filter_thermal(frame):
    return cv2.applyColorMap(frame, cv2.COLORMAP_HOT)

def filter_neon(frame):
    edges = cv2.Canny(frame, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def filter_sunset(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv[..., 0] = (hsv[..., 0] + 10) % 180  # Change hue
    hsv[..., 1] = hsv[..., 1] * 1.5  # Increase saturation
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def filter_paint(frame):
    return cv2.stylization(frame, sigma_s=60, sigma_r=0.07)

def filter_watercolor(frame):
    return cv2.stylization(frame, sigma_s=60, sigma_r=0.5)

def filter_film_noir(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def filter_solarize(frame):
    return cv2.threshold(frame, 128, 255, cv2.THRESH_BINARY_INV)[1]

def filter_bright_contrast(frame):
    return cv2.convertScaleAbs(frame, alpha=1.5, beta=50)

def filter_color_shift(frame):
    b, g, r = cv2.split(frame)
    return cv2.merge([r, g, b])

# List of filters
filters = [
    filter_original, filter_grayscale, filter_sepia, filter_invert,
    filter_blur, filter_edge_detection, filter_bright, filter_dark,
    filter_contrast, filter_vignette, filter_cartoon, filter_sketch,
    filter_hue, filter_pencil_sketch, filter_emboss, filter_solarize,
    filter_color_invert, filter_fog, filter_pixelate, filter_noise,
    filter_sharpen, filter_cinematic, filter_xray,
    filter_posterize, filter_color_pop, filter_film_grain, filter_glitch,
    filter_dreamy, filter_futuristic, filter_mosaic, filter_rainbow,
    filter_vintage, filter_smooth, filter_lomo, filter_saturation,
    filter_bokeh, filter_silhouette, filter_frosted, filter_thermal,
    filter_neon, filter_sunset, filter_paint, filter_watercolor,
    filter_film_noir, filter_bright_contrast, filter_color_shift
]

filter_names = [
    "Original", "Grayscale", "Sepia", "Invert", "Blur", "Edge Detection",
    "Bright", "Dark", "Contrast", "Vignette", "Cartoon", "Sketch",
    "Hue Change", "Pencil Sketch", "Emboss", "Solarize", "Color Invert",
    "Fog", "Pixelate", "Noise", "Sharpen", "Cinematic", "X-ray",
    "Posterize", "Color Pop", "Film Grain", "Glitch", "Dreamy",
    "Futuristic", "Mosaic", "Rainbow", "Vintage", "Smooth", "Lomo",
    "Saturation", "Bokeh", "Silhouette", "Frosted", "Thermal",
    "Neon", "Sunset", "Paint", "Watercolor", "Film Noir",
    "Bright Contrast", "Color Shift"
]

# Create a Tkinter window
root = tk.Tk()
root.title("Cinematic Filter")

# Global variable to hold the selected filter
selected_filter = StringVar(value=filter_names[0])

def update_frame():
    ret, frame = cap.read()
    if ret:
        # Apply the selected filter
        filter_index = filter_names.index(selected_filter.get())
        filtered_frame = filters[filter_index](frame)
        
        # Convert the frame to PhotoImage
        img = Image.fromarray(filtered_frame)
        imgtk = ImageTk.PhotoImage(image=img)
        
        # Update the label with the new image
        label.imgtk = imgtk
        label.configure(image=imgtk)
    
    # Call this function again after 10 milliseconds
    label.after(10, update_frame)

# Create a label to display the video
label = Label(root)
label.pack()

# Create a dropdown menu for filter selection
filter_menu = OptionMenu(root, selected_filter, *filter_names)
filter_menu.pack()

# Open a connection to the camera
cap = cv2.VideoCapture(0)

# Start updating the frames
update_frame()

# Start the Tkinter main loop
root.mainloop()

# Release the camera when the window is closed
cap.release()